# <center>**Trabajo Práctico Número 1**</center>

### <center>**Recorridos en Ecobicis - Año 2024**</center>

#### <center>Análisis de datos - CEIA, Fiuba</center>

---

**Integrantes del grupo**

- Espínola, Carla
- Gambarte, Antonella

---


### Bibliotecas


In [177]:
import pandas as pd

### Carga de datos desde archivo cvs


In [178]:
df_recorridos = pd.read_csv("../badata_ecobici_recorridos_realizados_2024.csv")
df_recorridos.head()

,id_recorrido,duracion_recorrido,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,id_usuario,modelo_bicicleta,genero
0,20428222,568,2024-01-23 18:36:00,513,308 - SAN MARTIN II,Av. San Martín 5129,-58.490739,-34.597130,2024-01-23 18:45:28,498,055 - HABANA,Gral. José Gervasio Artigas 4298 (y Habana),-58.494959,-34.586598,992557.0,FIT,MALE
1,20431744,1355,2024-01-23 22:41:20,460,133 - BEIRO Y SEGUROLA,Segurola 3194,-58.511930,-34.607500,2024-01-23 23:03:55,382,204 - Biarritz,Biarritz 2403,-58.477255,-34.605431,320782.0,FIT,FEMALE
2,20429936,0,2024-01-23 20:06:22,467,328 - SARMIENTO II,Sarmiento 2037,-58.395893,-34.605514,2024-01-23 20:06:22,6,006 - Parque Lezama,"Avenida Martin Garcia, 295",-58.369758,-34.628526,828678.0,FIT,FEMALE
3,20429976,0,2024-01-23 20:08:17,382,204 - Biarritz,Biarritz 2403,-58.477255,-34.605431,2024-01-23 20:08:17,460,133 - BEIRO Y SEGUROLA,Segurola 3194,-58.511930,-34.607500,320782.0,ICONIC,FEMALE
4,20424802,680,2024-01-23 15:18:39,137,137 - AZOPARDO Y CHILE,AZOPARDO 700,-58.367492,-34.615598,2024-01-23 15:29:59,150,150 - RODRIGO BUENO,Av. España 2200,-58.355465,-34.618755,861425.0,FIT,FEMALE


### Shape del dataframe y nombre de columnas


In [179]:
filas,columnas = df_recorridos.shape
print(f"El DataFrame tiene {filas} observaciones y {columnas} variables.")

El DataFrame tiene 3559284 observaciones y 17 variables.


In [180]:
print("Nombre de las variables:")

for i, col in enumerate(df_recorridos.columns):
    print(f"{i+1}. {col}")

Nombre de las variables:
1. id_recorrido
2. duracion_recorrido
3. fecha_origen_recorrido
4. id_estacion_origen
5. nombre_estacion_origen
6. direccion_estacion_origen
7. long_estacion_origen
8. lat_estacion_origen
9. fecha_destino_recorrido
10. id_estacion_destino
11. nombre_estacion_destino
12. direccion_estacion_destino
13. long_estacion_destino
14. lat_estacion_destino
15. id_usuario
16. modelo_bicicleta
17. genero


### Tipos de datos de cada variable


In [181]:
print(f"Columnas del dataframe:\n{df_recorridos.dtypes}\n")

Columnas del dataframe:
id_recorrido                    int64
duracion_recorrido              int64
fecha_origen_recorrido         object
id_estacion_origen              int64
nombre_estacion_origen         object
direccion_estacion_origen      object
long_estacion_origen          float64
lat_estacion_origen           float64
fecha_destino_recorrido        object
id_estacion_destino             int64
nombre_estacion_destino        object
direccion_estacion_destino     object
long_estacion_destino         float64
lat_estacion_destino          float64
id_usuario                    float64
modelo_bicicleta               object
genero                         object
dtype: object



### Para las fechas hacemos conversión


In [182]:
df_recorridos['fecha_destino_recorrido'] = pd.to_datetime(df_recorridos['fecha_destino_recorrido'])
df_recorridos['fecha_origen_recorrido'] = pd.to_datetime(df_recorridos['fecha_origen_recorrido'])

print(f"Columnas del dataframe:\n{df_recorridos.dtypes}\n")

Columnas del dataframe:
id_recorrido                           int64
duracion_recorrido                     int64
fecha_origen_recorrido        datetime64[ns]
id_estacion_origen                     int64
nombre_estacion_origen                object
direccion_estacion_origen             object
long_estacion_origen                 float64
lat_estacion_origen                  float64
fecha_destino_recorrido       datetime64[ns]
id_estacion_destino                    int64
nombre_estacion_destino               object
direccion_estacion_destino            object
long_estacion_destino                float64
lat_estacion_destino                 float64
id_usuario                           float64
modelo_bicicleta                      object
genero                                object
dtype: object



### Ahora comprobamos si existen observaciones duplicadas


In [183]:
print("Cantidad de datos duplicados:", df_recorridos.duplicated().sum())

Cantidad de datos duplicados: 1


In [184]:
duplicados = df_recorridos[df_recorridos.duplicated(keep=False)]
print("Datos de duplicado:\n")
duplicados.head()


Datos de duplicado:



,id_recorrido,duracion_recorrido,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,id_usuario,modelo_bicicleta,genero
434350,22425357,1119,2024-07-23 12:30:30,104,104 - Federico Lacroze,Av. Guzman 1100 & Av. Corrientes,-58.455212,-34.587617,2024-07-23 12:49:09,402,-- CDO Chacarita -- (Temporal),3209 Av. Francisco Beiro,-58.498542,-34.597612,1039158.0,FIT,MALE
434351,22425357,1119,2024-07-23 12:30:30,104,104 - Federico Lacroze,Av. Guzman 1100 & Av. Corrientes,-58.455212,-34.587617,2024-07-23 12:49:09,402,-- CDO Chacarita -- (Temporal),3209 Av. Francisco Beiro,-58.498542,-34.597612,1039158.0,FIT,MALE


### Existe solo un duplicado. Procedemos a eliminarlo.


In [185]:
df_recorridos = df_recorridos.drop_duplicates()
print("Duplicados después:", df_recorridos.duplicated().sum())

Duplicados después: 0


### Datos faltantes


In [186]:
print(f"Valores faltantes:\n{df_recorridos.isnull().sum()}") 

Valores faltantes:
id_recorrido                      0
duracion_recorrido                0
fecha_origen_recorrido            0
id_estacion_origen                0
nombre_estacion_origen            0
direccion_estacion_origen         0
long_estacion_origen              0
lat_estacion_origen               0
fecha_destino_recorrido        3379
id_estacion_destino               0
nombre_estacion_destino           0
direccion_estacion_destino        0
long_estacion_destino             0
lat_estacion_destino              0
id_usuario                        0
modelo_bicicleta                  0
genero                        11946
dtype: int64


Se observan datos faltantes en las variables fecha_destino_recorrido y genero.
Vamos a analizar primero los faltantes de género.

De las observaciones que conforman el dataset, armamos dos grupos. Uno con aquellos que contienen la variable genero completa y otro con aquellos que tienen ese campo vacío.


In [ ]:
con_genero = df_recorridos[df_recorridos['genero'].notnull()]
sin_genero = df_recorridos[df_recorridos['genero'].isnull()]

Primero analizamos los faltantes con respecto al modelo de bicicleta.


In [ ]:
print("Distribución modelo_bicicleta (con género):")
print(con_genero['modelo_bicicleta'].value_counts(normalize=True))

print("\nDistribución modelo_bicicleta (sin género):")
print(sin_genero['modelo_bicicleta'].value_counts(normalize=True))

Observamos que la distribución de modelo_bicicleta es prácticamente igual con y sin género. Entonces, el tipo de bicicleta no influye en los faltantes de género.


In [187]:
df_recorridos['genero'].unique()

array(['MALE', 'FEMALE', 'OTHER', nan], dtype=object)

In [188]:
df_recorridos['fecha_destino_recorrido'].unique()

<DatetimeArray>
['2024-01-23 18:45:28', '2024-01-23 23:03:55', '2024-01-23 20:06:22',
 '2024-01-23 20:08:17', '2024-01-23 15:29:59', '2024-01-23 17:55:51',
 '2024-01-23 21:37:14', '2024-01-23 22:35:32', '2024-01-23 19:55:40',
 '2024-01-23 21:39:37',
 ...
 '2024-04-25 12:57:57', '2024-04-25 10:05:57', '2024-04-26 01:14:26',
 '2024-04-25 17:00:11', '2024-04-26 08:55:37', '2024-04-25 21:43:59',
 '2024-04-25 17:50:40', '2024-04-25 18:09:14', '2024-04-25 19:37:29',
 '2024-04-25 13:29:32']
Length: 3220155, dtype: datetime64[ns]

In [189]:
df_recorridos['fecha_destino_recorrido'].value_counts(dropna=False)

fecha_destino_recorrido
NaT                    3379
2024-06-13 10:24:25      22
2024-08-07 14:00:18      16
2024-06-07 17:16:58      13
2024-07-26 10:07:32      12
                       ... 
2024-04-25 17:42:52       1
2024-04-26 08:26:47       1
2024-04-25 15:56:40       1
2024-04-26 08:06:18       1
2024-04-25 16:35:45       1
Name: count, Length: 3220155, dtype: int64

In [190]:
faltantes = df_recorridos[
    df_recorridos['genero'].isnull() | df_recorridos['fecha_destino_recorrido'].isnull()
]

faltantes.head(20)

,id_recorrido,duracion_recorrido,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,id_usuario,modelo_bicicleta,genero
242,20421678,1071,2024-01-23 10:36:25,23,023 - Suipacha,Suipacha & Viamonte,-58.379836,-34.600139,2024-01-23 10:54:16,172,172 - BRASIL,BRASIL&DEFENSA,-58.371082,-34.625426,22682.0,FIT,NaN
898,20430626,3011,2024-01-23 20:48:23,416,257 - Plaza Francisco Ramirez,"2700 Pedraza, Manuela",-58.469133,-34.553262,2024-01-23 21:38:34,318,285 - ESPINOSA,1785 Espinosa,-58.457317,-34.603936,20757.0,FIT,NaN
899,20428035,2754,2024-01-23 18:26:57,318,285 - ESPINOSA,1785 Espinosa,-58.457317,-34.603936,2024-01-23 19:12:51,416,257 - Plaza Francisco Ramirez,"2700 Pedraza, Manuela",-58.469133,-34.553262,20757.0,FIT,NaN
900,20421780,639,2024-01-23 10:47:46,242,103 - MALBA,Av. Pres. Figueroa Alcorta 3451,-58.403984,-34.576949,2024-01-23 10:58:25,14,014 - Pacifico,"Santa Fe Av. & Bullrich, Int. Av.",-58.426387,-34.577424,20753.0,ICONIC,NaN
1020,20422378,710,2024-01-23 11:43:34,59,018 - Independencia,Idependencia y Bernardo de Irigoyen,-58.380565,-34.617654,2024-01-23 11:55:24,83,083 - Paraná,1590 Lavalle,-58.389373,-34.603269,19535.0,FIT,NaN
1041,20423565,2100,2024-01-23 13:41:13,132,132 - CORRIENTES,Reconquista & Corrientes Av.,-58.372763,-34.603369,2024-01-23 14:16:13,32,032 - Catedral,Bartolomé Mitre & San Martín,-58.373656,-34.606745,19156.0,FIT,NaN
1518,23573179,786,2024-10-24 19:09:47,83,083 - Paraná,1590 Lavalle,-58.389373,-34.603269,2024-10-24 19:22:53,57,057 - BELGRANO,Balcarce 482,-58.370645,-34.613250,19535.0,FIT,NaN
2144,23567825,9,2024-10-24 09:39:19,93,093 - CARLOS CALVO,Sarandi 1010,-58.394464,-34.620798,2024-10-24 09:39:28,93,093 - CARLOS CALVO,Sarandi 1010,-58.394464,-34.620798,22001.0,FIT,NaN
2331,23556570,418,2024-10-23 09:44:29,93,093 - CARLOS CALVO,Sarandi 1010,-58.394464,-34.620798,2024-10-23 09:51:27,184,184 - Pedro Echague,Pedro Echague 1878 & Combate De Los Pozos,-58.391342,-34.630613,22001.0,FIT,NaN
2991,20408649,487,2024-01-22 13:18:33,152,152 - JULIETA LANTERI,"Vera Peñaloza, Rosario 599 & Lanteri Julieta",-58.359631,-34.618164,2024-01-22 13:26:40,111,111 - MACACHA GUEMES,Macacha Guemes 350,-58.364686,-34.605488,23524.0,FIT,NaN


In [191]:
faltantes = df_recorridos[df_recorridos.isnull().any(axis=1)]


faltantes.head()

,id_recorrido,duracion_recorrido,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,id_usuario,modelo_bicicleta,genero
242,20421678,1071,2024-01-23 10:36:25,23,023 - Suipacha,Suipacha & Viamonte,-58.379836,-34.600139,2024-01-23 10:54:16,172,172 - BRASIL,BRASIL&DEFENSA,-58.371082,-34.625426,22682.0,FIT,NaN
898,20430626,3011,2024-01-23 20:48:23,416,257 - Plaza Francisco Ramirez,"2700 Pedraza, Manuela",-58.469133,-34.553262,2024-01-23 21:38:34,318,285 - ESPINOSA,1785 Espinosa,-58.457317,-34.603936,20757.0,FIT,NaN
899,20428035,2754,2024-01-23 18:26:57,318,285 - ESPINOSA,1785 Espinosa,-58.457317,-34.603936,2024-01-23 19:12:51,416,257 - Plaza Francisco Ramirez,"2700 Pedraza, Manuela",-58.469133,-34.553262,20757.0,FIT,NaN
900,20421780,639,2024-01-23 10:47:46,242,103 - MALBA,Av. Pres. Figueroa Alcorta 3451,-58.403984,-34.576949,2024-01-23 10:58:25,14,014 - Pacifico,"Santa Fe Av. & Bullrich, Int. Av.",-58.426387,-34.577424,20753.0,ICONIC,NaN
1020,20422378,710,2024-01-23 11:43:34,59,018 - Independencia,Idependencia y Bernardo de Irigoyen,-58.380565,-34.617654,2024-01-23 11:55:24,83,083 - Paraná,1590 Lavalle,-58.389373,-34.603269,19535.0,FIT,NaN


## GENERO


In [192]:
faltantes_genero = df_recorridos[df_recorridos['genero'].isnull()]
faltantes_genero.head(20)

,id_recorrido,duracion_recorrido,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,id_usuario,modelo_bicicleta,genero
242,20421678,1071,2024-01-23 10:36:25,23,023 - Suipacha,Suipacha & Viamonte,-58.379836,-34.600139,2024-01-23 10:54:16,172,172 - BRASIL,BRASIL&DEFENSA,-58.371082,-34.625426,22682.0,FIT,NaN
898,20430626,3011,2024-01-23 20:48:23,416,257 - Plaza Francisco Ramirez,"2700 Pedraza, Manuela",-58.469133,-34.553262,2024-01-23 21:38:34,318,285 - ESPINOSA,1785 Espinosa,-58.457317,-34.603936,20757.0,FIT,NaN
899,20428035,2754,2024-01-23 18:26:57,318,285 - ESPINOSA,1785 Espinosa,-58.457317,-34.603936,2024-01-23 19:12:51,416,257 - Plaza Francisco Ramirez,"2700 Pedraza, Manuela",-58.469133,-34.553262,20757.0,FIT,NaN
900,20421780,639,2024-01-23 10:47:46,242,103 - MALBA,Av. Pres. Figueroa Alcorta 3451,-58.403984,-34.576949,2024-01-23 10:58:25,14,014 - Pacifico,"Santa Fe Av. & Bullrich, Int. Av.",-58.426387,-34.577424,20753.0,ICONIC,NaN
1020,20422378,710,2024-01-23 11:43:34,59,018 - Independencia,Idependencia y Bernardo de Irigoyen,-58.380565,-34.617654,2024-01-23 11:55:24,83,083 - Paraná,1590 Lavalle,-58.389373,-34.603269,19535.0,FIT,NaN
1041,20423565,2100,2024-01-23 13:41:13,132,132 - CORRIENTES,Reconquista & Corrientes Av.,-58.372763,-34.603369,2024-01-23 14:16:13,32,032 - Catedral,Bartolomé Mitre & San Martín,-58.373656,-34.606745,19156.0,FIT,NaN
1518,23573179,786,2024-10-24 19:09:47,83,083 - Paraná,1590 Lavalle,-58.389373,-34.603269,2024-10-24 19:22:53,57,057 - BELGRANO,Balcarce 482,-58.370645,-34.613250,19535.0,FIT,NaN
2144,23567825,9,2024-10-24 09:39:19,93,093 - CARLOS CALVO,Sarandi 1010,-58.394464,-34.620798,2024-10-24 09:39:28,93,093 - CARLOS CALVO,Sarandi 1010,-58.394464,-34.620798,22001.0,FIT,NaN
2331,23556570,418,2024-10-23 09:44:29,93,093 - CARLOS CALVO,Sarandi 1010,-58.394464,-34.620798,2024-10-23 09:51:27,184,184 - Pedro Echague,Pedro Echague 1878 & Combate De Los Pozos,-58.391342,-34.630613,22001.0,FIT,NaN
2991,20408649,487,2024-01-22 13:18:33,152,152 - JULIETA LANTERI,"Vera Peñaloza, Rosario 599 & Lanteri Julieta",-58.359631,-34.618164,2024-01-22 13:26:40,111,111 - MACACHA GUEMES,Macacha Guemes 350,-58.364686,-34.605488,23524.0,FIT,NaN


In [193]:
con_genero = df_recorridos[df_recorridos['genero'].notnull()]
sin_genero = df_recorridos[df_recorridos['genero'].isnull()]


In [194]:
print("Distribución modelo_bicicleta (con género):")
print(con_genero['modelo_bicicleta'].value_counts(normalize=True))

print("\nDistribución modelo_bicicleta (sin género):")
print(sin_genero['modelo_bicicleta'].value_counts(normalize=True))


Distribución modelo_bicicleta (con género):
modelo_bicicleta
FIT       0.725544
ICONIC    0.274456
Name: proportion, dtype: float64

Distribución modelo_bicicleta (sin género):
modelo_bicicleta
FIT       0.727775
ICONIC    0.272225
Name: proportion, dtype: float64


In [201]:
# con_genero['mes'] = con_genero['fecha_origen_recorrido'].dt.month
sin_genero = sin_genero.copy()
sin_genero['mes'] = sin_genero['fecha_origen_recorrido'].dt.month
print("Meses más frecuentes (sin género):")
print(sin_genero['mes'].value_counts())


Meses más frecuentes (sin género):
mes
10    1131
9     1090
5     1075
3     1072
11    1064
1      999
12     998
4      946
8      932
6      912
7      885
2      842
Name: count, dtype: int64


In [196]:
print("Estaciones origen con más faltantes de género:")
print(sin_genero['nombre_estacion_origen'].value_counts().head(10))


Estaciones origen con más faltantes de género:
nombre_estacion_origen
183 - VIRREY CEVALLOS             339
081 - PARQUE DE LAS CIENCIAS      248
054 - Acuña de Figueroa           199
162 - LARREA Y BARTOLOMÉ MITRE    192
218 - Campana                     178
027 - Montevideo                  168
093 - CARLOS CALVO                154
070 - ARAOZ                       153
087 - Guayaquil                   143
022 - Arenales                    140
Name: count, dtype: int64


---


In [197]:
faltantes_fecha = df_recorridos[df_recorridos['fecha_destino_recorrido'].isnull()]
faltantes_fecha.head(20)

,id_recorrido,duracion_recorrido,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,id_usuario,modelo_bicicleta,genero
63800,24502775,0,2024-12-31 20:34:50,324,280 - JURAMENTO,5209 Juramento,-58.484056,-34.578933,NaT,553,210 - JOSE LEON CABEZON,José León Cabezón 2516,-58.500942,-34.580192,594121.0,FIT,FEMALE
63801,24502746,0,2024-12-31 20:25:28,369,304 - Navarro,Navarro 2625,-58.491797,-34.592244,NaT,553,210 - JOSE LEON CABEZON,José León Cabezón 2516,-58.500942,-34.580192,1110710.0,ICONIC,MALE
64730,24493951,0,2024-12-30 21:08:03,418,345 - Plaza Mafalda,Conde & Concepción Arenal,-58.444580,-34.580938,NaT,418,345 - Plaza Mafalda,Conde & Concepción Arenal,-58.444580,-34.580938,1158360.0,ICONIC,FEMALE
64741,24493039,0,2024-12-30 19:55:50,385,268 - Plaza Aristóbulo Del Valle,Campana & Marcos Sastre,-58.493141,-34.606160,NaT,498,055 - HABANA,Gral. José Gervasio Artigas 4298 (y Habana),-58.494959,-34.586598,443254.0,ICONIC,MALE
64748,24493560,0,2024-12-30 20:33:54,14,014 - Pacifico,"Santa Fe Av. & Bullrich, Int. Av.",-58.426387,-34.577424,NaT,387,278 - Donato Alvarez,"1540 Garcia, Juan Agustin",-58.465568,-34.602028,1174804.0,FIT,FEMALE
64777,24493742,0,2024-12-30 20:49:11,135,135 - MARCELO T. DE ALVEAR,Esmeralda & Paraguay,-58.378363,-34.597642,NaT,60,060 - 25 De Mayo,527 25 De Mayo,-58.371079,-34.601651,1179399.0,FIT,MALE
64798,24494592,0,2024-12-30 22:35:52,261,272 - Plaza Bruno Giordano,Bruno Giordano&Neuquen,-58.440584,-34.616151,NaT,182,182 - RAVIGNANI Y GUATEMALA,"2185 Ravignani, Emilio, Dr. & Guatemala",-58.435247,-34.578048,1126152.0,FIT,FEMALE
64799,24494340,0,2024-12-30 21:54:20,568,342 - HONDURAS,Honduras 5190,-58.431960,-34.587550,NaT,182,182 - RAVIGNANI Y GUATEMALA,"2185 Ravignani, Emilio, Dr. & Guatemala",-58.435247,-34.578048,911217.0,FIT,FEMALE
64800,24494597,0,2024-12-30 22:36:35,261,272 - Plaza Bruno Giordano,Bruno Giordano&Neuquen,-58.440584,-34.616151,NaT,182,182 - RAVIGNANI Y GUATEMALA,"2185 Ravignani, Emilio, Dr. & Guatemala",-58.435247,-34.578048,1126150.0,ICONIC,MALE
64806,24493656,0,2024-12-30 20:42:15,478,015 - AMENABAR,Teodoro García 2613,-58.447524,-34.572056,NaT,361,258 - Plaza J.J. Paso,1399 Moldes,-58.453608,-34.569187,327145.0,FIT,FEMALE


In [198]:
faltantes_por_estacion = faltantes_fecha.groupby('id_estacion_destino').size().reset_index(name='cantidad_faltantes')

faltantes_por_estacion.head(40)

,id_estacion_destino,cantidad_faltantes
0,3,3
1,8,16
2,9,11
3,12,1
4,13,17
5,14,7
6,17,64
7,21,18
8,24,12
9,25,103


In [199]:
faltantes_fecha = df_recorridos[df_recorridos['fecha_destino_recorrido'].isnull()]

faltantes_por_estacion = (
    faltantes_fecha
    .groupby(['id_estacion_destino', 'nombre_estacion_destino'])
    .size()
    .reset_index(name='cantidad_faltantes')
    .sort_values(by='cantidad_faltantes', ascending=False)
    .reset_index(drop=True)
)

faltantes_por_estacion.head(20)

,id_estacion_destino,nombre_estacion_destino,cantidad_faltantes
0,121,121 - YATAY,147
1,25,025 - Plaza Guemes,103
2,69,069 - Ecuador,75
3,144,144 - PUEYRREDÓN,71
4,54,054 - Acuña de Figueroa,68
5,275,256 - PLAZA NORUEGA,68
6,193,193 - ARENALES Y AGUERO,65
7,17,017 - Plaza Almagro,64
8,184,184 - Pedro Echague,60
9,199,199 - ESTADOS UNIDOS Y BOEDO,59
